In [1]:
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
colors = [[0.8,0,0],[0,0.8,0],[0,0,0.8],[0,0.8,0.8],[0.8,0.8,0.8]]
def get_vis_tip(grasps):
    vis_sps = []
    for i in range(len(grasps)):
        vis_sp = o3d.geometry.TriangleMesh.create_sphere(0.01)
        vis_sp.paint_uniform_color(colors[i])
        vis_sp.translate(grasps[i,:3])
        vis_sps.append(vis_sp)
    return vis_sps

def project_grasp(tip_poses,kd_tree, points, normals):
    grasps = []
    for i, tip_pose in enumerate(tip_poses):
        idx = kd_tree.search_knn_vector_3d(tip_pose, 1)[1]
        grasp = np.hstack([points[idx], normals[idx]])
        grasps.append(grasp)
    return grasps

In [3]:
plate_mesh = o3d.io.read_triangle_mesh("../envs/assets/plate_cvx.ply")
plate_mesh.compute_vertex_normals()
plate_mesh.compute_triangle_normals()
plate_pcd = plate_mesh.sample_points_poisson_disk(2048, use_triangle_normal=True)
kd_tree = o3d.geometry.KDTreeFlann(plate_pcd)
points = np.asarray(plate_pcd.points)
normals = np.asarray(plate_pcd.normals)

In [12]:
o3d.visualization.draw_geometries_with_vertex_selection([plate_pcd])

[Open3D INFO] Clearing all points from selection.
[Open3D INFO] Adding point #1275 (0.11, -0.01, 0.02) to selection.
[Open3D INFO] No points have been picked.
[Open3D INFO] Adding point #1400 (0.10, 0.00, 0.01) to selection.
[Open3D INFO] Adding point #865 (0.10, -0.01, 0.01) to selection.
[Open3D INFO] Adding point #1107 (0.10, -0.03, 0.01) to selection.
[Open3D INFO] Adding point #771 (0.10, -0.05, 0.02) to selection.


In [14]:
ps = np.vstack([np.array([[0.11693159157663433, -0.0010921809379487205, 0.03147002839388472],[0.11639388645545473, 0.018195489236448273, 0.012567558375736702]]),points[[865,1107,771]]])
ns = normals[[1275, 1400, 865, 1107, 771]]
grasp = np.hstack([ps,ns])

In [15]:
vis_tips = get_vis_tip(grasp)
o3d.visualization.draw_geometries([plate_pcd]+vis_tips)

In [24]:
o3d.io.write_point_cloud("pose_9.ply",plate_pcd)

True

In [27]:
grasp = grasp.reshape(1,5,6)

In [16]:
np.save("../data/seeds_plate/grasps/pose_9.npy",grasp.reshape(1,5,6))
np.save("../data/predicted_grasps/debug_50.0.npy",grasp)